In [ ]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Download NLTK resources
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# ---------- Preprocessing ----------
def preprocess_dataframe(df):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    def clean_text(text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", "", text)
        text = re.sub(r'\d+', '', text)
        words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
        return ' '.join(words)

    df['content'] = df['title'].fillna('') + ' ' + df['text'].fillna('')
    df['cleaned_content'] = df['content'].apply(clean_text)
    df = df[df['cleaned_content'].str.split().str.len() > 5]

    label_cols = ['label', 'is_fake', 'fake', 'target']
    label_column = next((col for col in label_cols if col in df.columns), df.columns[0])
    df['label'] = df[label_column]

    return df

# load and preprocess
df = pd.read_csv('fake_real_news.csv')
df = preprocess_dataframe(df)

# machine learning 
X = df['cleaned_content']
y = df['label']
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
X_feature = vectorizer.fit_transform(X)

if len(vectorizer.vocabulary_) == 0:
    print("Feature extraction didn't happen.")
else:
    print("Feature extraction done.")

X_train, X_test, y_train, y_test = train_test_split(X_feature, y, test_size=0.2, random_state=42)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

plt.figure(figsize=(8, 6))
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"\n{name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Traditional Models")
plt.legend()
plt.show()

# Lstm feature extraction
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_content'])
sequences = tokenizer.texts_to_sequences(df['cleaned_content'])
X_lstm = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')
y_lstm = df['label'].values

X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# lstm model
lstm_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("\nTraining LSTM model...")
history = lstm_model.fit(X_lstm_train, y_lstm_train, epochs=3, batch_size=64, validation_split=0.2, verbose=1)


lstm_preds = lstm_model.predict(X_lstm_test).flatten()
lstm_preds_class = (lstm_preds > 0.5).astype(int)

print("\nLSTM Model")
print("Accuracy:", accuracy_score(y_lstm_test, lstm_preds_class))
print("Precision:", precision_score(y_lstm_test, lstm_preds_class))
print("Recall:", recall_score(y_lstm_test, lstm_preds_class))
print("F1 Score:", f1_score(y_lstm_test, lstm_preds_class))
print("Confusion Matrix:\n", confusion_matrix(y_lstm_test, lstm_preds_class))

fpr, tpr, _ = roc_curve(y_lstm_test, lstm_preds)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve for LSTM
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f'LSTM (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - LSTM Model")
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'